In [1]:
from selenium import webdriver
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

In [2]:
sys.dont_write_bytecode = True
sys.path.append('../')
from tools.maneger import DataManager

In [4]:
# """
# 起動1
# java -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role hub

# 起動2
# java -Dwebdriver.chrome.driver="C:\Users\mmi-lab\Documents\selenium\chromedriver.exe" -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role node -hub http://192.168.1.224:4444

# """

# """
# 使えそう
# ジャンルからフレーズ・例文を探す
# https://gogakuru.com/english/phrase/genre/index.html

# """
print("aa")

aa


In [17]:
class Crawler:
    def __init__(self, ) -> None:
        self.YN_stop_list = "Is Do Did Can Have".split()
        self.WH_stop_list = "What When Where Who What Why How".split()
        self.please_stop_list = ["please", "Would you", "Could you"]
        self.proposal_stop_list = ["Let's", "Shall we", "Why don't you", "Why don't we"]
        self.plain_stop_list = "I She He They We You".split()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def crawl(self, mode, num):
        self.mode = mode
        self.num = num
        if mode == "YN":
            self.stop_list = self.YN_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode == "WH":
            self.stop_list = self.WH_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode =="please":
            self.stop_list = self.please_stop_list
            self.essential = "?"
            self._crawl_ENG()
        
        elif mode =="proposal":
            self.stop_list = self.proposal_stop_list
            self.essential = "?"
            self._crawl_ENG()

        elif mode == "plain":
            self.stop_list = self.plain_stop_list
            self.essential = "."
            self._crawl_ENG()
        
    
    def _in(self, search, target):
        """
        search : list or str
        target : str
        """
        result = True
        if isinstance(search, list):
            for s in search:
                result = result and s in target
            return result
        elif isinstance(search, str):
            return search in target
        else:
            print("list or str is expected, but not")
            return False
    
    def _extract_stop_type(self, target):
        if self.essential not in target:
            return None
        for stop in self.stop_list:
            if stop in target:
                return stop
        return None
    
    def _extract_label(self, text):
        if self.mode == "YN":
            return ["YN", self._extract_stop_type(text)]
        # 後で
        elif self.mode == "WH":
            return ["WH", self._extract_stop_type(text)]
        elif self.mode == "plain":
            return ["plain"]
        elif self.mode == "proposal":
            return ["proposal", self._extract_stop_type(text)]
        else:
            return ["please"]
    
    def _go_conv_page(self):
        element = self.driver.find_element_by_class_name("condition").find_elements_by_tag_name("a")
        element[1].click()
        
    def _crawl_ENG(self):
        self.data = []
        self.label = []

        self.driver.get("https://gogakuru.com/english/phrase/genre/index.html")
        
        self._go_conv_page()

        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        while 1:
            cells = soup.find_all("td", class_="summary")
            for cell in cells:
                engs = cell.find_all("span", class_="font-en")
                text_ = [eng.get_text() for eng in engs]
                text = text_[0]
                # print(text)
                # 大文字で含まれるならば
                stop = self._extract_stop_type(text)
                if  stop:
                    jp = cell.find("dd", class_="jp").get_text()
                    self.data.append(jp)
                    self.label.append( self._extract_label(text) )
                    print("stop:{0}, jp:{1}".format(stop, jp))

            # 一度打ち切り
            if len(self.data) >= self.num:
                break
        
            # 次のurl
            try:
                next_t = self.driver.find_element_by_class_name("right").find_element_by_tag_name('a')
                next_t.click()
            except:
                break
            html = self.driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
        

    def make_corpus(self):
        self.corpus = {}
        self.corpus[self.mode] = []
        for t, l in zip(self.data, self.label):
            one_data = {"data":t, "label":l}
            self.corpus[self.mode].append(one_data)
    
    def write(self, filename):
        with open(filename, "w") as f:
            json.dump(self.corpus, f,  ensure_ascii=False, indent=4)
    
    def quit(self):
        self.driver.quit() 



In [12]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path
dataM = DataManager(data_path)

In [18]:
crawler = Crawler()
mode = "proposal"

corpus_name = "{0}.json".format(mode)

crawler.connect("192.168.1.224")

In [19]:
crawler.crawl(mode,300)

stop:Shall we, jp:そろそろインタビューを始めようか？
stop:Shall we, jp:そろそろ家に帰ろうか？
stop:Why don't we, jp:アイスクリームを食べない？
stop:Why don't we, jp:向こうで雨宿りしない？
stop:Shall we, jp:チケットを取りましょうか？
stop:Shall we, jp:教授の肩に強力なＮマークをつけますか、閣下？
stop:Why don't you, jp:あなたの落語をご両親に見てもらうのはどうでしょう？
stop:Why don't you, jp:スープを作ったら？
stop:Why don't you, jp:サヤ、あなたから始めてくれる？
stop:Why don't you, jp:「夕食を用意する間に、お風呂に入ったらどうですか？」
stop:Why don't you, jp:公園に行って遊んできたらどう？
stop:Shall we, jp:そうしましょうか？
stop:Why don't you, jp:マンガを描いたらどうかしら？
stop:Why don't we, jp:しばらくここにいてはどうかな？
stop:Why don't you, jp:また働き始めたら？
stop:Why don't we, jp:あの自動車修理工場に行ってみよう。
stop:Why don't we, jp:彼女に助けを頼もうか？
stop:Shall we, jp:そろそろ行こうか？
stop:Shall we, jp:休憩しようか？
stop:Why don't we, jp:みんなで行ってご挨拶しないか。
stop:Shall we, jp:外に行かない？
stop:Let's, jp:えーと…あの…デービッドと母さん？
stop:Why don't you, jp:あとで返事をすればいいじゃない。
stop:Why don't you, jp:おばあさんに電話してあげたらどうなの？
stop:Why don't you, jp:どうして雪おろしのボランティアを募集しないの？
stop:Why don't you, jp:思い切って外に出て、もう少し日本を見てみたらどう？
stop:Why don't you, jp:もう少しここにいたらどう？
sto

In [20]:
crawler.make_corpus()
crawler.write(data_path+corpus_name)


In [21]:
crawler.quit()